In [24]:
# import the dependencies
import pandas as pd
import numpy as np
import pymongo

In [25]:
df = pd.read_csv("r-joined.csv")
df.drop(columns=[df.columns.tolist()[0]], inplace=True)
df.shape

(8098, 3177)

In [26]:
df.head()

,CASEID,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,OLDP2FWT,P2WTV2,PVT,PVTRT,P2WTV3.y,EMP,EMPST,EMPW,EMPH,EMPOTH
0,10001,60080192,1,8128,9,28,90,9,28,90,...,NaN,NaN,1,0.6016,NaN,2,0,1,0,0
1,10002,16030048,1,3941,10,1,90,10,1,90,...,NaN,NaN,0,2.1321,NaN,2,0,1,0,0
2,10003,74070592,1,7813,10,5,90,10,5,90,...,NaN,NaN,0,1.3077,NaN,2,0,1,0,0
3,10004,74030720,1,7813,10,10,90,10,10,90,...,NaN,NaN,1,0.7391,NaN,3,0,0,1,0
4,10005,4180080,1,9018,10,10,90,10,10,90,...,NaN,NaN,0,1.2613,NaN,2,0,1,0,0


In [27]:
DS2_diagnosis_variables = [name for name in df.columns if 'LT' in name]

In [28]:
DS1_cat_attribs = ['V5114', 'V5118', 'V5113', 'V5115', 'V5112', 'V4428',\
                   'V4433', 'V6305', 'V6301', 'V5225', 'V6649', 'V6749',\
                   'V5918', 'V6143', 'V6126', 'V6114', 'V101', 'V102', 'V6215']
DS2_cat_attribs = ['SEX', 'EDCAT', 'RACE', 'MARSTAT', 'REL', 'EMP'] + DS2_diagnosis_variables
DS1_response = ['V6309']
DS2_num_attribs = ['AGE']

# Code the variables

In [29]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [30]:
# Define databases and collections - one for each dataset
ds2_db = client.ncs1_ds2_db
ds2_collection = ds2_db.variable_maps
ds1_db = client.ncs_db
ds1_collection = ds1_db.variable_maps

In [31]:
def convert_labels_in_place (df, var_name, db_collection):
    var_map = db_collection.find_one({'Name': var_name})['map']
    new_map = {int(key):value for key,value in var_map.items()}
    df[var_name] = df[var_name].map(new_map)
    print(df[var_name].value_counts())

In [32]:
for var in DS1_cat_attribs + DS1_response:
    convert_labels_in_place(df, var, ds1_collection)

A LITTLE      2579
NA            2171
SOMEWHAT      1548
NOT AT ALL    1340
VERY           391
INAP            67
DK               2
Name: V5114, dtype: int64
NOT AT ALL    2871
A LITTLE      2217
NA            2180
SOMEWHAT       629
VERY           131
INAP            67
DK               3
Name: V5118, dtype: int64
A LITTLE      2828
NA            2163
SOMEWHAT      1342
NOT AT ALL    1294
VERY           404
INAP            67
Name: V5113, dtype: int64
A LITTLE      3056
NA            2165
SOMEWHAT      1287
NOT AT ALL    1273
VERY           249
INAP            67
DK               1
Name: V5115, dtype: int64
A LITTLE      2624
NA            2155
SOMEWHAT      1717
NOT AT ALL    1047
VERY           486
INAP            67
DK               2
Name: V5112, dtype: int64
NA                 2188
A LITTLE TRUE      2170
SOMEWHAT TRUE      1397
NOT TRUE AT ALL    1353
VERY TRUE           936
INAP                 53
DK                    1
Name: V4428, dtype: int64
NOT TRUE AT ALL    2725
NA    

In [33]:
for var in DS2_cat_attribs:
    convert_labels_in_place(df, var, ds2_collection)

Female    4263
Male      3835
Name: SEX, dtype: int64
12 Years       2679
13-15 Years    2132
16+ Years      1813
0-11 Years     1474
Name: EDCAT, dtype: int64
White       6084
Black       1011
Hispanic     733
Other        270
Name: RACE, dtype: int64
Marcoh    4410
Marnev    2435
Marswd    1253
Name: MARSTAT, dtype: int64
Protestant             4475
Catholic               2250
No Preference, None     780
Other                   593
Name: REL, dtype: int64
Working, incl. temp. laid off, matern./s    5979
Student                                      840
Homemaker                                    719
Other                                        560
Name: EMP, dtype: int64
Absent     6177
Present    1921
Name: ALCALT1, dtype: int64
Absent     7317
Present     781
Name: ALCALT2, dtype: int64
Absent     6886
Present    1212
Name: ALCDLT, dtype: int64
Absent     7154
Present     944
Name: DRGALT1, dtype: int64
Absent     7712
Present     386
Name: DRGALT2, dtype: int64
Absent     7469
Pre

In [34]:
columns_to_keep = ['CASEID'] + DS1_response + DS1_cat_attribs + DS2_num_attribs + DS2_cat_attribs
#columns_to_keep

In [35]:
rdf = df[columns_to_keep]
rdf.shape

(8098, 56)

In [36]:
rdf = rdf.query('V6309 != "INAP" & V6309 != "NA"')

In [37]:
rdf.shape

(5872, 56)

In [38]:
rdf.to_csv("reduced-data2.csv", index=False)